In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE
import gradio as gr
import numpy as np

df = pd.read_csv("BMW_Car_Sales_Classification.csv")
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')
print("Class distribution:\n", df['sales_classification'].value_counts())
print("\nFeature distributions for Sold (High) class:")
sold_df = df[df['sales_classification'] == 'High']
for col in ['price_usd', 'mileage_km', 'engine_size_l', 'year']:
    print(f"{col}: mean={sold_df[col].mean():.2f}, min={sold_df[col].min()}, max={sold_df[col].max()}")
df['sales_classification'] = df['sales_classification'].map({'Low': 0, 'High': 1})
y = df['sales_classification']
X = df.drop(columns=['sales_classification', 'sales_volume'])

cat_cols = X.select_dtypes(include='object').columns.tolist()
num_cols = X.select_dtypes(exclude='object').columns.tolist()
preprocessor = ColumnTransformer([
    ("cat", OneHotEncoder(handle_unknown='ignore'), cat_cols),
    ("num", StandardScaler(), num_cols)
])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train_preprocessed = preprocessor.fit_transform(X_train)
X_test_preprocessed = preprocessor.transform(X_test)
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_preprocessed, y_train)
clf = RandomForestClassifier(n_estimators=200, max_depth=10, min_samples_split=5, random_state=42)
clf.fit(X_train_resampled, y_train_resampled)

y_pred = clf.predict(X_test_preprocessed)
print("\nClassification Report:\n", classification_report(y_test, y_pred, target_names=["Not Sold", "Sold"]))
feature_names = preprocessor.named_transformers_['cat'].get_feature_names_out(cat_cols).tolist() + num_cols
importances = clf.feature_importances_
print("\nFeature Importances:")
for feature, importance in sorted(zip(feature_names, importances), key=lambda x: x[1], reverse=True):
    print(f"{feature}: {importance:.4f}")

import gradio as gr

def predict_sale(model, year, region, color, fuel_type, transmission, engine_size, mileage, price):
    try:
        sample = pd.DataFrame([{
            "model": model,
            "year": year,
            "region": region,
            "color": color,
            "fuel_type": fuel_type,
            "transmission": transmission,
            "engine_size_l": engine_size,
            "mileage_km": mileage,
            "price_usd": price
        }])
        sample_preprocessed = preprocessor.transform(sample)
        prob = clf.predict_proba(sample_preprocessed)[0][1]
        result = clf.predict(sample_preprocessed)[0]
        return f"Prediction: {'Car Likely to be SOLD' if result == 1 else 'Car Likely NOT to be Sold'}"
    except Exception as e:
        return f"Error: {str(e)}"

models = df['model'].unique().tolist()
regions = ["Asia", "Middle East", "Europe", "North America", "Africa", "South America"]
colors = ["Red", "Silver", "Black", "White", "Blue", "Grey"]
fuel_types = ["Petrol", "Diesel", "Hybrid", "Electric"]
transmissions = ["Manual", "Automatic"]

with gr.Blocks(css="""
body {
    background-color: #252C37; /* Midnight */
    color: #F8C61E; /* Sunburst */
    font-family: 'Inter', sans-serif;
    margin: 0;
    padding: 0;
}
.gradio-container {
    max-width: 100%;
    margin: 0;
    padding: 40px 20px;
    background: #252C37; /* Midnight */
    min-height: 100vh;
}
h1 {
    font-size: 2.5em;
    font-weight: 700;
    color: #F8C61E; /* Sunburst */
    text-align: center;
    margin-bottom: 10px;
}
h3 {
    font-size: 1.2em;
    font-weight: 400;
    color: #C0C0C0; /* Adjusted for readability */
    text-align: center;
    margin-bottom: 30px;
}
.gr-row {
    display: flex;
    gap: 30px;
    flex-wrap: wrap;
    max-width: 1200px;
    margin: 0 auto;
}
.gr-column {
    flex: 1;
    min-width: 300px;
}
.gr-accordion {
    background: #1A2A44; /* Darker tone for contrast */
    border: 1px solid #4B5EAA;
    border-radius: 8px;
    padding: 15px;
}
.gr-accordion .label {
    background: #F8C61E !important; /* Sunburst */
    color: #252C37 !important; /* Midnight */
    font-weight: 600;
    border-radius: 6px;
    padding: 10px;
}
.gr-dropdown, .gr-slider, .gr-textbox {
    background: #1A2A44; /* Darker tone for contrast */
    border: 1px solid #4B5EAA;
    border-radius: 6px;
    padding: 10px;
    color: #FFFFFF; /* White for better readability */
    font-size: 1em;
}
.gr-slider .slider {
    accent-color: #F8C61E; /* Sunburst */
}
button {
    background: #F8C61E !important; /* Sunburst */
    color: #252C37 !important; /* Midnight */
    font-size: 1.1em !important;
    font-weight: 600;
    border-radius: 6px !important;
    padding: 12px 24px !important;
    border: none !important;
    transition: background 0.3s ease;
}
button:hover {
    background: #e6b015 !important; /* Slightly darker sunburst for hover */
    box-shadow: 0 2px 8px rgba(0, 0, 0, 0.1);
}
.output-textbox textarea {
    color: #F8C61E; /* Sunburst */
    font-size: 1.2em;
    font-weight: 500;
    background: #1A2A44; /* Darker tone for contrast */
    border: 1px solid #4B5EAA;
    border-radius: 6px;
    padding: 15px;
    text-align: center;
}
.header-container {
    padding: 20px 0;
    border-bottom: 1px solid #4B5EAA;
    margin-bottom: 40px;
}
.footer {
    text-align: center;
    color: #A0AEC0; /* Lighter grey for footer */
    font-size: 0.9em;
    margin-top: 40px;
}
@media (max-width: 768px) {
    .gr-row {
        flex-direction: column;
        gap: 20px;
    }
    .gr-column {
        min-width: 100%;
    }
}
""") as app:

    with gr.Group(elem_classes=["header-container"]):
        gr.HTML("""
        <h1>Bayerische Motoren Werke Sales Prediction Dashboard</h1>
        <h3>An AI-powered BMW sales prediction system that analyzes key vehicle attributes — such as model, year, region, mileage, and price — to determine the likelihood of a successful sale. Designed to provide sellers and dealerships with actionable insights before listing.</h3>
        """)

    with gr.Row():
        with gr.Column():
            with gr.Accordion("Car Details", open=True):
                model_input = gr.Dropdown(choices=models, label="Car Model", value="3 Series")
                region_input = gr.Dropdown(choices=regions, label="Region", value="Europe")
                color_input = gr.Dropdown(choices=colors, label="Color", value="White")
            with gr.Accordion("Specifications", open=True):
                fuel_input = gr.Dropdown(choices=fuel_types, label="Fuel Type", value="Petrol")
                transmission_input = gr.Dropdown(choices=transmissions, label="Transmission", value="Automatic")
                year_input = gr.Slider(2010, 2024, step=1, label="Year", value=2018)

        with gr.Column():
            with gr.Accordion("Pricing & Usage", open=True):
                engine_input = gr.Slider(1.5, 5.0, step=0.1, label="Engine Size (L)", value=3.2)
                mileage_input = gr.Slider(0, 200000, step=1000, label="Mileage (KM)", value=100000)
                price_input = gr.Slider(10000, 200000, step=1000, label="Price (USD)", value=75000)
            predict_btn = gr.Button("Predict Sale")
            output = gr.Textbox(label="Prediction Result", elem_classes=["output-textbox"])

    gr.HTML("<div class='footer'>Powered by xAI | BMW Sales Prediction Model</div>")

    predict_btn.click(
        fn=predict_sale,
        inputs=[model_input, year_input, region_input, color_input, fuel_input, transmission_input, engine_input, mileage_input, price_input],
        outputs=output
    )

app.launch(share=False, debug=True)

Class distribution:
 sales_classification
Low     34754
High    15246
Name: count, dtype: int64

Feature distributions for Sold (High) class:
price_usd: mean=74966.82, min=30000, max=119997
mileage_km: mean=100882.82, min=21, max=199938
engine_size_l: mean=3.24, min=1.5, max=5.0
year: mean=2017.05, min=2010, max=2024

Classification Report:
               precision    recall  f1-score   support

    Not Sold       0.70      0.96      0.81      6968
        Sold       0.30      0.04      0.08      3032

    accuracy                           0.68     10000
   macro avg       0.50      0.50      0.44     10000
weighted avg       0.58      0.68      0.59     10000


Feature Importances:
year: 0.1037
engine_size_l: 0.0833
model_X1: 0.0566
mileage_km: 0.0549
model_X6: 0.0479
price_usd: 0.0471
model_7 Series: 0.0434
model_i3: 0.0427
model_M3: 0.0385
model_M5: 0.0382
model_X5: 0.0374
model_i8: 0.0358
model_3 Series: 0.0352
model_X3: 0.0307
color_Grey: 0.0266
region_South America: 0.0236
model

<IPython.core.display.Javascript object>